In [2]:
from typing import ForwardRef
import torch
from torch import nn
from torch._C import import_ir_module
import torch.nn.functional as F
import torchvision
import os

from resnet_4ch import resnet

class Config(object):
    pretrained_model_path = './pretrained_models'

    backbone = 'resnet18'
    class_num = 2
    without_mask =  True
    embed_size = 64 # 不确定
    loss_pcl = 10

opt = Config()

In [ ]:
resnet_layers = int(opt.backbone.split('resnet')[-1]) #34
backbone = resnet(resnet_layers,
                        True,
                        os.path.join(opt.pretrained_model_path, opt.backbone+'.pth'))
# drop pool layer and fc layer, resnet34 layer4 output shape: b,512,8,8
features = list(backbone.children())
len(features)
#features[6]
#features[7]
backbone = nn.Sequential(*features[:7])
prediction_head = nn.Sequential(*features[7])

img = torch.rand(4,3,256,256)
tmp = backbone(img)
tmp = prediction_head(tmp)


In [ ]:

#a=nn.AvgPool1d

features[-3]

In [13]:
embedding_layer = nn.Conv2d(256,opt.embed_size,kernel_size=(1,1))
x = torch.rand(4,256,16,16)
x = embedding_layer(x)
x.shape 

torch.Size([4, 64, 16, 16])

In [20]:
import numpy as np
x = torch.rand(4,64,16,16)
y = torch.zeros(size=(16,16,4,16,16))
for i in range(16):
    for j in range(16):
        y[i,j,:,:,:] = torch.sum(x[:,:,i,j,None,None] * x,dim=1)/np.sqrt(opt.embed_size)

y.shape       
y = torch.sigmoid(y)


 



torch.Size([16, 16, 4, 16, 16])